In [ ]:
###
# Importar librerias necesarias para el procesamiento del dataframe
###

'''Pandas para la lectura del archivo'''
import pandas as pd

'''Sns y plt para organizar los graficos visuales'''
import seaborn as sns
import matplotlib.pyplot as plt

###
# Importar librerias necesarias del modelo, el entrenamiendo y las métricas
###

'''Train test split para dividir el data frame en partes'''
from sklearn.model_selection import train_test_split

'''Preprocesamiento de datos'''
from sklearn.preprocessing import StandardScaler, OneHotEncoder

'''Transformadores de columnas para procesar datos'''
from sklearn.compose import ColumnTransformer

'''Modelo de ejecución de entrenamiento'''
from sklearn.linear_model import LogisticRegression

'''Metricas de desempeño'''
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

'''Pipelines para transformar los datos procesados y el modelo'''
from sklearn.pipeline import Pipeline

#
# Importar el archivo csv del set online llamado car heart_cleveland_upload.csv
#
df = pd.read_csv('/content/sample_data/heart_cleveland_upload.csv')

#
# 1. Para este caso de estudio todas las variables tienen relación y se
# trabajaran con todas en el procesamiento y entremaniento del modelo
#

#
# 2. Preprocesamiento y Selección de Características
#
features = df.drop('condition', axis=1)
target = df['condition']

#
# Preprocesamiento para variables numéricas y categóricas
#
numeric_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

#
# 3. Selección de caracteristicas relevantes
#
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

#
# 4. Dividir el conjunto de datos
#
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#
# Creación del pipeline de regresión logística
#
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

#
# 5. Entrenamiento del modelo
#
model.fit(X_train, y_train)

#
# 6. Evaluación del desempeño del modelo
#
y_pred = model.predict(X_test)

#
# Predecir probabilidades para el conjunto de test
#
y_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba)

print(f"Auc Score: {auc}")

#
# Imprimir el reporte de clasificación
#
print(classification_report(y_test, y_pred))

#
# 7. Generar y mostrar la matriz de confusión y grafica adicional ROC
#
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

#
# Calcular la curva ROC
#
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

#
# Graficar la curva ROC
#
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Dashed diagonal
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()